In [1]:
!pip install openai langchain-community langchain-chroma langchain langchain-text-splitters langchain_core langchain_huggingface pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.0/329.0 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.2/490.2 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 79.6 MB/s eta 0:00:

In [7]:
from openai import OpenAI
import re
from langchain_core.prompts import PromptTemplate

In [8]:
BASE_URL = "https://openrouter.ai/api/v1"
OPEN_API_KEY = "OpenRouter_Api-Key".........................................API_KEY
# MODEL = "mistralai/devstral-2512:free"
MODEL = "xiaomi/mimo-v2-flash:free"

In [9]:
client=OpenAI(
    base_url=BASE_URL,
    api_key=OPEN_API_KEY
)

In [10]:
template_1 = """
You are an Artificial Intelligence Assistant.

Your role is to receive a user query, analyze it carefully, and generate a detailed,
step-by-step response based on the query.

Rules and Instructions:
1. Information must be factual and accurate.
2. Do not generate harmful, offensive, or misleading content.
3. Prefer the latest and most relevant information when possible.
4. The response should be clean, clear, and free from unnecessary special characters.

User Query:
{input}
"""


template_2 = """
You are an Artificial Intelligence Assistant working as a text summarizer.

Your role is to receive text from the user, analyze it, and generate a concise summary
in clear bullet points highlighting the main ideas.

Rules and Instructions:
1. Information must be factual and accurate.
2. Do not generate harmful, offensive, or misleading content.
3. Prefer the latest and most relevant information when possible.
4. The summary should be clean, concise, and free from unnecessary special characters.

Input Text:
{input}
"""


template_3 = """
You are an Artificial Intelligence Assistant working as a detailed explainer.

Your role is to receive a summary in bullet points from the user and expand each point
into a clear, detailed explanation step by step.

Rules and Instructions:
1. Information must be factual and accurate.
2. Do not generate harmful, offensive, or misleading content.
3. Prefer the latest and most relevant information when possible.
4. The explanation should be clean, structured, and free from unnecessary special characters.

Input Summary:
{input}
"""


list_of_templates = [template_1, template_2, template_3]


In [11]:
# ---------------- PROMPT MAKING ----------------
def prompt_making(user_input, template):
    prompt = PromptTemplate(
        input_variables=["input"],
        template=template
    )
    return prompt.format(input=user_input)


# ---------------- RESPONSE GENERATION ----------------
def response_generation(final_input):
    response_obj = client.responses.create(
        model=MODEL,
        input=final_input,
        max_output_tokens=1024,
        temperature=0.7,
        extra_headers={
            "HTTP-Referer": "https://company-chatbot.local",
            "X-Title": "Company RAG Chatbot",
        }
    )

    # Extract text
    text = response_obj.output_text

    # Clean unnecessary characters (safe cleaning)
    clean_text = re.sub(r'[##_]+', '', text)

    return clean_text.strip()


# ---------------- PROMPT CHAINING ----------------
def prompt_chaining(initial_input, templates):
    response = initial_input

    for idx, template in enumerate(templates, start=1):
        final_prompt = prompt_making(response, template)
        response = response_generation(final_prompt)

        print(f"\n--- Response after Template {idx} ---\n")
        print(response)

    return response

In [12]:
final_output = prompt_chaining(
    initial_input="Pakistan neiguber countries and there capital",
    templates=list_of_templates
)


--- Response after Template 1 ---

Here is a list of Pakistan's neighboring countries and their capitals.

Pakistan shares land borders with **four** countries: India, Afghanistan, Iran, and China. It also has a maritime border with Oman.

**1. India**
*   **Capital:** New Delhi

**2. Afghanistan**
*   **Capital:** Kabul

**3. Iran**
*   **Capital:** Tehran

**4. China**
*   **Capital:** Beijing

**5. Oman** (Maritime Neighbor)
*   **Capital:** Muscat

***Note:** While Pakistan and Tajikistan do not share a land border, they are separated only by a narrow strip of land known as the **Wakhan Corridor** (in Afghanistan).*

--- Response after Template 2 ---

Here is a summary of Pakistan's neighboring countries and their capitals:

*   **India:** Capital is New Delhi
*   **Afghanistan:** Capital is Kabul
*   **Iran:** Capital is Tehran
*   **China:** Capital is Beijing
*   **Oman:** Capital is Muscat (shares a maritime border)

**Note:** Pakistan shares a land border with four countries.